In [1]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter, HTMLConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from cStringIO import StringIO


def convert_pdf_to_txt(path, pages):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = HTMLConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = file(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()

    for i, page in enumerate(PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True)):
        interpreter.process_page(page)
        if i >= pages:
            break

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    return text

path = '/home/zhongzhu/Desktop/Project2/Test/Seattle.pdf'
f = open('/home/zhongzhu/Desktop/Project2/Test/pos_extracted0801', 'w')
f.write(convert_pdf_to_txt(path,100))
f.close()

In [1]:
import re

fp = '/home/zhongzhu/Desktop/Project2/Test/pos_extracted2'
f = open(fp, 'r')
boxes = f.read().split('TEXT_BOX')
f.close()

contents = {}
ls = []
for content in boxes:
    c = content.split(';')
    if(c[0].find('left:') > 0): #if not empty
        l = c[0][c[0].find('left:')+5:c[0].find('px')]
        t = c[1][c[1].find('top:')+4:c[1].find('px')]
        w = c[2][c[2].find('width:')+6:c[2].find('px')]
        h = c[3][c[3].find('height:')+7:c[3].find('px')]
        
        text = "".join(c[4:])
        lines = text.split('\n')
        
        tempCount = 0
        for line in lines:
            if not line in contents:
                contents[line.decode('utf-8')] = []
                ls.append(line.decode('utf-8'))
                #line = line + " *"
            if re.search(r'[A-Za-z0-9,.?!]', line):
                contents[line.decode('utf-8')].append( (int(l), int(t)+tempCount, int(w), int(h)) )
            tempCount += 1
#print contents
for i, text in enumerate(contents):
    if i > 100:
        break
    print(text+ ';')
    print(contents[text])
    print('----')

;
[]
----
 DATE;
[(281, 1593, 16, 8), (508, 1593, 16, 8)]
----
 No accidents reported today;
[(85, 1033, 109, 10)]
----
 No preparatory or initial inspections were held today;
[(85, 462, 202, 10)]
----
 No QC requirements were completed today;
[(85, 522, 167, 10)]
----
Asbestos Abatement;
[(95, 642, 92, 37)]
----
 No QC Punch List items were issued today;
[(85, 553, 164, 24)]
----
plans and specifications, to the best of my knowledge, except as noted above.;
[(183, 1558, 386, 33)]
----
 Page 1 of 2;
[(494, 113, 38, 10)]
----
 ACTIVITY START/FINISH;
[(75, 479, 89, 10)]
----
Worked;
[(508, 632, 26, 20)]
----
Held demo prep meeting today with AAI.  See attached minutes of meeting.  Will start next week with demo.  They brought new ;
[(95, 397, 427, 27)]
----
 Hours;
[(508, 631, 26, 20)]
----
 CONTRACTORS ON SITE (Report first and/or last day contractors were on site);
[(75, 583, 276, 10)]
----
 REMARKS;
[(95, 328, 59, 18)]
----
DATE;
[(417, 114, 16, 22), (417, 956, 16, 22)]
----
 No activ

In [4]:
#for i, text in enumerate(contents):
#   if len(contents[text]) > 800:
#      print(text+ ' ' + str(len(contents[text]))+' '+str(contents[text][0]))
   #     print('--------------------------')
        
import csv
f = open('empty_table.csv', 'w')
writer = csv.writer(f)

for i, text in enumerate(ls):
    if len(contents[text]) > 30:
        print text
        writer.writerow([text, len(contents[text]), contents[text][0]])
f.close()

 REPORT NUMBER
DATE
 Page 1 of 2
 CONTRACT NUMBER
 DACA67-02-D-2012  0004
 WEATHER
Weather Caused No Delay
 CONTRACTORS QUALITY CONTROL REPORT (QCR)
 DAILY LOG OF CONSTRUCTION - MILITARY
 PROJECT
 D/B Replace Military Family Housing P5
 CONTRACTOR
 Evergreene Construction, L.C.
 QC NARRATIVES
 Activities in Progress:
 AAI
 General Comments:
None today
 QUALITY CONTROL TESTING AND RESULTS
 REMARKS
No remarks today
 Safety Inspection / Safety Meetings:
 DAILY SAFETY INSPECTIONS
 PREPARATORY AND/OR INITIALS HELD TODAY
 PREP/INITIAL DATES
 (Preparatory and initial dates held and advance notice)
 No preparatory or initial inspections were held today
 ACTIVITY START/FINISH
 No activities were started or finished today
 QC REQUIREMENTS
 No QC requirements were completed today
 QA/QC PUNCH LIST
 (Describe QC Punch List items issued, Report QC and QA Punch List items corrected)
 No QC Punch List items were issued today
No Punch List items were corrected today
 No contractors had their first or 

In [5]:
from itertools import product
neighbors = {}
    
for line1, line2 in product(contents, contents):
    if(line1 != line2
       and abs(int(contents[line1][0])-int(contents[line2][0]))<30
       and abs(int(contents[line1][1])-int(contents[line2][1]))<30):
        key = line1
        value = line2
        if not key in neighbors:
            neighbors[key] = []
        neighbors[key].append(value)

IndexError: list index out of range

In [4]:
import Tkinter as tk
import ttk

win = tk.Tk()
win.title("Set Pairs")   

ttk.Label(win, text="Select category:").grid(column=0, row=0)   
ttk.Label(win, text="Bind content").grid(column=2, row=0)      


def select():
    dropdown2['values'] = neighbors[category.get()]  
    dropdown2.current(0)    
    
# selecte button
action = ttk.Button(win, text="Select", command=select)
action.grid(column=1, row=1)   

pairs = {}
# 
def bind():   
    key = contents[category.get()]
    new_value = contents[content.get()]
    if not key in pairs:
        pairs[key] = set([])
    pairs[key].add(new_value)
    dropdown2.current(0) 
    

# bind button
action = ttk.Button(win, text="Bind", command=bind)
action.grid(column=3, row=1) 

# creat a drop-down list
category = tk.StringVar()
dropdown1 = ttk.Combobox(win, width=60, height=20, textvariable=category)
dropdown1['values'] = list(neighbors.keys())     
dropdown1.grid(column=0, row=1)      
#dropdown1.current(0) 

# create another drop-down list
content = tk.StringVar()
dropdown2 = ttk.Combobox(win, width=60, textvariable=content)    
dropdown2.grid(column=2, row=1)        

win.mainloop()      # window

In [5]:
for i in pairs:
    print i
    print pairs[i]

(417, 114, 16, 22)
set([(424, 136, 76, 10)])
(417, 106, 56, 8)
set([(417, 113, 16, 22)])
(319, 174, 203, 28)
set([(319, 176, 203, 28), (319, 175, 203, 28)])


In [6]:
path = '/home/zhongzhu/Desktop/Project2/Test/Seattle.pdf'
f = open('/home/zhongzhu/Desktop/Project2/Test/pos_extracted2', 'w')
f.write(convert_pdf_to_txt(path,2026))
f.close()

# extract text and its position

In [1]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter, HTMLConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from cStringIO import StringIO


def convert_pdf_to_txt(path1, path2, num):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = HTMLConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = file(path1, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()

    for i, page in enumerate(PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True)):
        interpreter.process_page(page)
        if i >= num:
            break
    text = retstr.getvalue()
    file_name = 'page' + str(num)
    f = open(path2+file_name, 'w')
    f.write(text)
    f.close()
    #text = retstr.getvalue()
    fp.close()
    device.close()
    retstr.close()



In [2]:
path1 = '/home/zhongzhu/Desktop/Project2/Test/Seattle.pdf'
path2 = '/home/zhongzhu/Desktop/Project2/Test/temp/'
convert_pdf_to_txt(path1, path2, 500)

# record positions

In [3]:
import re

def record_pos(path, page):
    f = open(path2+'page'+str(page), 'r')
    boxes = f.read().split('TEXT_BOX')
    f.close()

    contents = {}
    for content in boxes:
        c = content.split(';')
        if(c[0].find('left:') > 0): #if not empty
            l = c[0][c[0].find('left:')+5:c[0].find('px')]
            t = c[1][c[1].find('top:')+4:c[1].find('px')]
            w = c[2][c[2].find('width:')+6:c[2].find('px')]
            h = c[3][c[3].find('height:')+7:c[3].find('px')]

            text = "".join(c[4:])
            lines = text.split('\n')
                
            tempCount = 0
            for line in lines:
                if re.search(r'[A-Za-z0-9,.?!]', line):
                    if not line.decode('utf-8') in contents:
                        contents[line.decode('utf-8')] = [(int(l), int(t)+tempCount, int(w), int(h))]
                    else:
                        contents[line.decode('utf-8')].append((int(l), int(t)+tempCount, int(w), int(h)))
                tempCount += 1
    return contents


In [4]:
contents = record_pos(path2, 0)
for pos in contents:
    print(pos)
    print(contents[pos])

 No preparatory or initial inspections were held today
[(85, 462, 202, 10)]
 No QC requirements were completed today
[(85, 522, 167, 10)]
Asbestos Abatement
[(95, 642, 92, 37)]
 No QC Punch List items were issued today
[(85, 553, 164, 24)]
 Page 1 of 2
[(494, 113, 38, 10)]
 ACTIVITY START/FINISH
[(75, 479, 89, 10)]
Worked
[(508, 632, 26, 20)]
Held demo prep meeting today with AAI.  See attached minutes of meeting.  Will start next week with demo.  They brought new 
[(95, 397, 427, 27)]
 Hours
[(508, 631, 26, 20)]
 CONTRACTORS ON SITE (Report first and/or last day contractors were on site)
[(75, 583, 276, 10)]
 REMARKS
[(95, 328, 59, 18)]
DATE
[(417, 114, 16, 22)]
 No activities were started or finished today
[(85, 492, 165, 10)]
 CONTRACTORS QUALITY CONTROL REPORT (QCR)
[(123, 110, 227, 12)]
 (Describe QC Punch List items issued, Report QC and QA Punch List items corrected)
[(170, 540, 290, 10)]
 The following labor hours were Reported today:
[(85, 628, 189, 50)]
 DACA67-02-D-2012  000

# search neighbors

In [5]:
from itertools import product
neighbors = {}
    
for line1, line2 in product(contents, contents):
    if(line1 != line2
       and abs(int(contents[line1][0][0])-int(contents[line2][0][0]))<30
       and abs(int(contents[line1][0][1])-int(contents[line2][0][1]))<30):
        key = line1
        value = line2
        if not key in neighbors:
            neighbors[key] = []
        neighbors[key].append(value)

# GUI

In [6]:
import Tkinter as tk
import ttk

win = tk.Tk()
win.title("Set Pairs")   

ttk.Label(win, text="Select category:").grid(column=0, row=0)   
ttk.Label(win, text="Bind content").grid(column=2, row=0)      


def select():
    dropdown2['values'] = neighbors[category.get()]  
    dropdown2.current(0)    
    
# selecte button
action = ttk.Button(win, text="Select", command=select)
action.grid(column=1, row=1)   

pairs = {}
# 
def bind():   
    #key = contents[category.get()]
    #new_value = contents[content.get()]
    
    key = category.get()
    a = contents[category.get()][0]
    b = contents[content.get()][0]
    new_value = (b[0]-a[0], b[1]-a[1], b[2]-a[2], b[3]-a[3])
    
    if not key in pairs:
        pairs[key] = set([])
    pairs[key].add(new_value)
    dropdown2.current(0) 
    

# bind button
action = ttk.Button(win, text="Bind", command=bind)
action.grid(column=3, row=1) 

# creat a drop-down list
category = tk.StringVar()
dropdown1 = ttk.Combobox(win, width=60, height=20, textvariable=category)
dropdown1['values'] = list(neighbors.keys())     
dropdown1.grid(column=0, row=1)      
#dropdown1.current(0) 

# create another drop-down list
content = tk.StringVar()
dropdown2 = ttk.Combobox(win, width=60, textvariable=content)    
dropdown2.grid(column=2, row=1)        

win.mainloop()      # window

In [7]:
for i in pairs:
    print i
    print pairs[i]

DATE
set([(7, 22, 60, -12)])
 WEATHER
set([(0, 2, 0, 0), (0, 1, 0, 0)])


In [10]:
def find_content(content_map, content_pos):
    
    content = []
    for c in content_map:
        m = 1 #radius
        for pos in content_map[c]:
            error = abs(content_pos[0]-pos[0]) + abs(content_pos[1]-pos[1])
            #print error
            if error < m:
                content.append(c)
    #print(m)
    return content
    
data = {}
content_map = record_pos(path2, 500)
for cat1, cat2 in product(content_map, pairs):
    if cat1 == cat2:
        for pos, shift in product(content_map[cat1], pairs[cat2]):
            content_pos = (pos[0]+shift[0], pos[1]+shift[1], pos[2]+shift[2], pos[3]+shift[3])
            content = find_content(content_map, content_pos)
            if cat1 in data:
                data[cat1].append(content)
            else:
                data[cat1] = [content]
           


In [11]:
print len(data[' WEATHER'])
for i in data:
    print i
    for j in data[i]:
        print j


408
DATE
[u' 01 Nov 2004 - Monday']
[u' 01 Nov 2004 - Monday']
[u' 02 Nov 2004 - Tuesday']
[u' 02 Nov 2004 - Tuesday']
[u' 03 Nov 2004 - Wednesday']
[u' 03 Nov 2004 - Wednesday']
[u' 04 Nov 2004 - Thursday']
[u' 04 Nov 2004 - Thursday']
[u' 05 Nov 2004 - Friday']
[u' 05 Nov 2004 - Friday']
[u' 06 Nov 2004 - Saturday']
[u' 08 Nov 2004 - Monday']
[u' 09 Nov 2004 - Tuesday']
[u' 09 Nov 2004 - Tuesday']
[u' 10 Nov 2004 - Wednesday']
[u' 10 Nov 2004 - Wednesday']
[u' 11 Nov 2004 - Thursday']
[u' 11 Nov 2004 - Thursday']
[u' 12 Nov 2004 - Friday']
[u' 12 Nov 2004 - Friday']
[u' 15 Nov 2004 - Monday']
[u' 15 Nov 2004 - Monday']
[u' 16 Nov 2004 - Tuesday']
[u' 16 Nov 2004 - Tuesday']
[u' 17 Nov 2004 - Wednesday']
[u' 17 Nov 2004 - Wednesday']
[u' 18 Nov 2004 - Thursday']
[u' 18 Nov 2004 - Thursday']
[u' 19 Nov 2004 - Friday']
[u' 19 Nov 2004 - Friday']
[u' 22 Nov 2004 - Monday']
[u' 22 Nov 2004 - Monday']
[u' 23 Nov 2004 - Tuesday']
[u' 23 Nov 2004 - Tuesday']
[u' 24 Nov 2004 - Wednesday']
[u'

# test

In [1]:
for i in content_map:
    print i
    print content_map[i]

NameError: name 'content_map' is not defined

In [94]:
len(data[' WEATHER'])

1410

In [62]:
contents = record_pos(path2, 14)
for pos in contents:
    print(pos)
    print(contents[pos])

 Employer * * * * * *
(95, 10799, 71, 23)
None today * * * * * * * * * * * * * * * *
(95, 5474, 175, 18)
 PREP/INITIAL DATES * * * * *
(75, 8658, 75, 10)
 The following activity was started today:
(85, 10613, 156, 10)
 No contractors had their first or last day on site today * * * * * * *
(85, 10755, 206, 10)
 WEATHER * * * * * *
(319, 9436, 229, 28)
 QUALITY CONTROL INSPECTIONS AND RESULTS * * *
(95, 7013, 168, 18)
 DACA67-02-D-2012  0004 * * * * * * * * * * * * *
(424, 11105, 89, 10)
DATE * * * * *
(417, 4324, 16, 22)
None today * * * * * * * * * * * * *
(95, 5330, 168, 18)
Worked *
(508, 2721, 26, 20)
0.0 * * * *
(441, 2722, 36, 44)
 Evergreene Construction, L.C. * * * * * * *
(82, 11970, 102, 10)
 LABOR HOURS * * * * * *
(75, 9859, 53, 10)
 LABOR HOURS
(75, 614, 53, 10)
 No accidents reported today * *
(85, 4445, 109, 10)
 CONTRACTORS ON SITE (Report first and/or last day contractors w ere on site) * * * * * *
(75, 10741, 278, 10)
 General Comments:
(85, 267, 78, 10)
 QA/QC PUNCH L